In [1]:
#!pip install geopy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
#from geopy.geocoders import Nominatim
import datetime

In [4]:
# Define the path to the folder
# folder_path = Path("C:/Users/crazy/OneDrive - The City College of New York/DSE I2100 - Applied Machine Learning and Data Mining/Project")
# csv_file = folder_path.glob("*.csv").__next__()

# Load CSV file into DataFrame
# df = pd.read_csv(csv_file)


# df = pd.read_csv('/content/drive/MyDrive/Motor_Vehicle_Collisions_-_Crashes.csv', nrows=400000)

df = pd.read_csv('/content/drive/MyDrive/Motor_Vehicle_Collisions_-_Crashes.csv')


<ipython-input-4-4dca41c7a111>:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Motor_Vehicle_Collisions_-_Crashes.csv')


## Data Preprocessing

In [5]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2077598 entries, 0 to 2077597
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   CRASH DATE                     2077598 non-null  object 
 1   CRASH TIME                     2077598 non-null  object 
 2   BOROUGH                        1431167 non-null  object 
 3   ZIP CODE                       1430917 non-null  object 
 4   LATITUDE                       1843830 non-null  float64
 5   LONGITUDE                      1843830 non-null  float64
 6   LOCATION                       1843830 non-null  object 
 7   ON STREET NAME                 1636440 non-null  object 
 8   CROSS STREET NAME              1292051 non-null  object 
 9   OFF STREET NAME                348785 non-null   object 
 10  NUMBER OF PERSONS INJURED      2077580 non-null  float64
 11  NUMBER OF PERSONS KILLED       2077567 non-null  float64
 12  NUMBER OF PEDE

In [6]:
df.isnull().sum()

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           646431
ZIP CODE                          646681
LATITUDE                          233768
LONGITUDE                         233768
LOCATION                          233768
ON STREET NAME                    441158
CROSS STREET NAME                 785547
OFF STREET NAME                  1728813
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       6814
CONTRIBUTING FACTOR VEHICLE 2     322220
CONTRIBUTING FACTOR VEHICLE 3    1929123
CONTRIBUTING FACTOR VEHICLE 4    2044064
CONTRIBUTING FACTOR VEHICLE 5    2068510
COLLISION_ID                           0
VEHICLE TYPE COD

### Correcting values in # of persons injured/killed:

In [7]:
#Sum the injury-related columns
injury_columns = ['NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF MOTORIST INJURED']
df['NUMBER OF PERSONS INJURED'] = df[injury_columns].sum(axis=1)

#Sum the killed-related columns
killed_columns = ['NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED']
df['NUMBER OF PERSONS KILLED'] = df[killed_columns].sum(axis=1)

### The Following code replaces row value coordinates not in NYC with NaN for 'LATITUDE' & 'LONGITUDE' columns:

In [8]:
# Approximate coordinates for New York City:
# Maximum Latitude: 40.9176 (Northernmost point of the Bronx)
# Minimum Latitude: 40.4774 (Southernmost point of Staten Island)
# Maximum Longitude: -73.7004 (Easternmost point of Queens)
# Minimum Longitude: -74.2591 (Westernmost point of Staten Island)

# Define the maximum and minimum values for latitude and longitude
max_latitude = 40.9176
min_latitude = 40.4774
max_longitude = -73.7004
min_longitude = -74.2591

# Filter the DataFrame based on the conditions for latitude and longitude
invalid_latitudes = (df['LATITUDE'] > max_latitude) | (df['LATITUDE'] < min_latitude)
invalid_longitudes = (df['LONGITUDE'] > max_longitude) | (df['LONGITUDE'] < min_longitude)

# Replace the values with NaN where the conditions are not met
df.loc[invalid_latitudes, 'LATITUDE'] = np.nan
df.loc[invalid_longitudes, 'LONGITUDE'] = np.nan

### Creating target column "CLASS TYPE":

In [9]:
def determine_class_type(row):
    if row['NUMBER OF PERSONS INJURED'] == 0 and row['NUMBER OF PERSONS KILLED'] == 0:
        return '0'
    elif 1 <= row['NUMBER OF PERSONS INJURED'] <= 5 and row['NUMBER OF PERSONS KILLED'] == 0:
        return '1'
    elif row['NUMBER OF PERSONS INJURED'] > 5 and row['NUMBER OF PERSONS KILLED'] == 0:
        return '2'
    elif 1 <= row['NUMBER OF PERSONS KILLED'] <= 3:
        return '3'
    elif row['NUMBER OF PERSONS KILLED'] > 3:
        return '4'

df['CLASS TYPE'] = df.apply(determine_class_type, axis=1)
df['CLASS TYPE'] = df['CLASS TYPE'].astype('category')

### Dropping redundant/unnecessary columns:

In [10]:
df.drop(columns = ['BOROUGH', 'ZIP CODE', 'LOCATION', 'ON STREET NAME','CROSS STREET NAME','OFF STREET NAME',
                   'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
                   'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
                   'NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED', 'COLLISION_ID'], axis=1, inplace=True)

In [11]:
df.sample(30)

,CRASH DATE,CRASH TIME,LATITUDE,LONGITUDE,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CLASS TYPE
1496432,08/16/2014,21:42,40.876535,-73.870747,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN,0
2044196,12/04/2023,23:00,NaN,NaN,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,Sedan,NaN,NaN,NaN,NaN,0
1303333,07/13/2015,12:40,40.825578,-73.918460,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,0
1897223,09/07/2012,17:31,40.659932,-73.891648,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN,0
1871433,09/17/2012,17:00,40.755207,-73.996654,Unspecified,Unspecified,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,0
22295,05/21/2021,13:55,40.664467,-73.768310,Backing Unsafely,Unspecified,NaN,NaN,NaN,Pick-up Truck,Sedan,NaN,NaN,NaN,0
937620,02/14/2017,13:00,40.709866,-73.816420,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,NaN,NaN,NaN,NaN,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,0
2034602,11/21/2023,17:00,40.701614,-73.914856,Passing Too Closely,NaN,NaN,NaN,NaN,Sedan,NaN,NaN,NaN,NaN,0
572789,09/13/2018,14:18,40.883484,-73.901596,Backing Unsafely,Unspecified,NaN,NaN,NaN,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,0
604747,07/28/2018,16:40,40.737198,-73.708990,Unspecified,Unspecified,NaN,NaN,NaN,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,0


In [12]:
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].str.title()
df['VEHICLE TYPE CODE 2'] = df['VEHICLE TYPE CODE 2'].str.title()
df['VEHICLE TYPE CODE 3'] = df['VEHICLE TYPE CODE 3'].str.title()
df['VEHICLE TYPE CODE 4'] = df['VEHICLE TYPE CODE 4'].str.title()
df['VEHICLE TYPE CODE 5'] = df['VEHICLE TYPE CODE 5'].str.title()



### Removing/Replacing NaN values for Contributing Factor & Vehicle Type columns:

In [13]:
# Drop NaN values from subset columns
df = df.dropna(subset=['CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1'])

# List of column pairs
column_pairs = [
    ('CONTRIBUTING FACTOR VEHICLE 2', 'VEHICLE TYPE CODE 2'),
    ('CONTRIBUTING FACTOR VEHICLE 3', 'VEHICLE TYPE CODE 3'),
    ('CONTRIBUTING FACTOR VEHICLE 4', 'VEHICLE TYPE CODE 4'),
    ('CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 5')
]

# Iterate over each column pair
for factor_column, type_column in column_pairs:
    # Check if factor_column has a value and type_column is NaN
    mask = (pd.notna(df[factor_column])) & (pd.isna(df[type_column]))
    # Check if type_column has a value and factor_column is NaN
    mask |= (pd.notna(df[type_column])) & (pd.isna(df[factor_column]))
    # Drop rows where either condition is met
    df = df[~mask]

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

# Fill NaN values in contributing factor vehicle columns with 'No vehicle'
for factor_column, type_column in column_pairs:
    df[factor_column].fillna('None', inplace=True)
    df[type_column].fillna('None', inplace=True)

### Dropping rows with NaN values:

In [14]:
df = df.dropna()

### Verifying column dtype:

In [15]:
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M').dt.time
df['LATITUDE'] = df['LATITUDE'].astype('float64')
df['LONGITUDE'] = df['LONGITUDE'].astype('float64')
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].astype('object')
df['CONTRIBUTING FACTOR VEHICLE 2'] = df['CONTRIBUTING FACTOR VEHICLE 2'].astype('object')
df['CONTRIBUTING FACTOR VEHICLE 3'] = df['CONTRIBUTING FACTOR VEHICLE 3'].astype('object')
df['CONTRIBUTING FACTOR VEHICLE 4'] = df['CONTRIBUTING FACTOR VEHICLE 4'].astype('object')
df['CONTRIBUTING FACTOR VEHICLE 5'] = df['CONTRIBUTING FACTOR VEHICLE 5'].astype('object')
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].astype('object')
df['VEHICLE TYPE CODE 2'] = df['VEHICLE TYPE CODE 2'].astype('object')
df['VEHICLE TYPE CODE 3'] = df['VEHICLE TYPE CODE 3'].astype('object')
df['VEHICLE TYPE CODE 4'] = df['VEHICLE TYPE CODE 4'].astype('object')
df['VEHICLE TYPE CODE 5'] = df['VEHICLE TYPE CODE 5'].astype('object')

In [16]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1701713 entries, 3 to 1927466
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   CRASH DATE                     1701713 non-null  datetime64[ns]
 1   CRASH TIME                     1701713 non-null  object        
 2   LATITUDE                       1701713 non-null  float64       
 3   LONGITUDE                      1701713 non-null  float64       
 4   CONTRIBUTING FACTOR VEHICLE 1  1701713 non-null  object        
 5   CONTRIBUTING FACTOR VEHICLE 2  1701713 non-null  object        
 6   CONTRIBUTING FACTOR VEHICLE 3  1701713 non-null  object        
 7   CONTRIBUTING FACTOR VEHICLE 4  1701713 non-null  object        
 8   CONTRIBUTING FACTOR VEHICLE 5  1701713 non-null  object        
 9   VEHICLE TYPE CODE 1            1701713 non-null  object        
 10  VEHICLE TYPE CODE 2            1701713 non-null  object    

### Keeping only top 10 values in Contributing Factor & Vehicle Type columns:

In [17]:
# List of columns to iterate through
columns_to_iterate = [
    'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
    'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'
]

# Iterate over each column
for column in columns_to_iterate:
    # Get the top 10 most frequent values in the column
    top_10_values = df[column].value_counts().head(10).index.tolist()
    # Replace values not in top 10 with a placeholder value
    df[column] = df[column].apply(lambda x: x if x in top_10_values else 'Other')

    # Drop rows where any of the specified columns have 'Other' value
    df = df[df[column] != 'Other']

    # Reset the index after dropping rows
    df.reset_index(drop=True, inplace=True)


In [18]:
df.sample(35)

,CRASH DATE,CRASH TIME,LATITUDE,LONGITUDE,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CLASS TYPE
183863,2019-09-10,08:50:00,40.657490,-73.893074,Unspecified,Unspecified,None,None,None,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,0
833942,2014-05-16,10:46:00,40.654221,-73.923848,Unspecified,Unspecified,None,None,None,Sport Utility / Station Wagon,Passenger Vehicle,None,None,None,1
865457,2014-01-11,17:59:00,40.650655,-73.952953,Failure to Yield Right-of-Way,None,None,None,None,Passenger Vehicle,None,None,None,None,1
945179,2013-05-30,16:30:00,40.618973,-74.005227,Unspecified,Unspecified,None,None,None,Sport Utility / Station Wagon,Sport Utility / Station Wagon,None,None,None,1
733864,2015-04-21,17:08:00,40.738552,-73.999682,Unspecified,Unspecified,None,None,None,Passenger Vehicle,Taxi,None,None,None,0
689232,2015-09-20,16:15:00,40.643146,-73.901466,Driver Inattention/Distraction,Unspecified,None,None,None,Passenger Vehicle,Passenger Vehicle,None,None,None,0
1072719,2023-09-21,16:00:00,40.714455,-73.935295,Passing or Lane Usage Improper,Unspecified,None,None,None,Box Truck,Sedan,None,None,None,0
588558,2016-09-26,08:25:00,40.734590,-73.864620,Driver Inattention/Distraction,Unspecified,None,None,None,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,0
297172,2018-11-06,15:30:00,40.704525,-73.928406,Passing or Lane Usage Improper,Unspecified,None,None,None,Box Truck,Sedan,None,None,None,0
853898,2014-03-15,21:00:00,40.744099,-73.995644,Fatigued/Drowsy,Unspecified,None,None,None,Passenger Vehicle,Taxi,None,None,None,0


### Converting new dataframe to csv file:

In [19]:
# Save the DataFrame to a CSV file
df.to_csv('Processed_Data_v2.csv', index=False)